In [1]:
import mdtraj as md
from itertools import combinations, product
import numpy as np
import matplotlib.pyplot as plt
import os

def calculate_rdf(traj, non_bond=True, min_cutoff=0, max_cutoff=16):
    num_bins=20*max_cutoff
    
    # Select atoms of type A and type B
    atoms_A = traj.topology.select("name == A")
    atoms_B = traj.topology.select("name == B")
    
    # Generate pairs for A-A, B-B, and A-B interactions
    pairs_AA = np.array(list(combinations(atoms_A, 2)))  # A-A pairs
    pairs_BB = np.array(list(combinations(atoms_B, 2)))  # B-B pairs
    pairs_AB = np.array(list(product(atoms_A, atoms_B)))   # A-B pairs
    
    # Combine all pairs into one array if necessary, or you can calculate separately
    pairs_dict = {
        'AA': pairs_AA,
        'BB': pairs_BB,
        'AB': pairs_AB
    }

    CG_rdf = {}

    # Filter non-bonded pairs if required
    if non_bond:
        bonds = set([tuple(sorted((bond[0].index, bond[1].index))) for bond in traj.topology.bonds])

        # Filter for each type of pair
        for pair_type, pairs in pairs_dict.items():
            nonbonded_pairs = np.array([pair for pair in pairs if tuple(sorted(pair)) not in bonds])
            pairs_dict[pair_type] = nonbonded_pairs

    # Calculate RDF for each pair type
    for pair_type, pairs in pairs_dict.items():
        r, g_r = md.compute_rdf(traj, pairs=pairs, r_range=(min_cutoff/10, max_cutoff/10), n_bins=num_bins)
        CG_rdf[pair_type] = (r, g_r)

    return CG_rdf

In [2]:
topology_psf_path = "structure.psf"
AA_trajectory_data_path = "CG.lammpstrj"
CG_trajectory_data_path = "AA.lammpstrj"


AA_traj = md.load(AA_trajectory_data_path, top=topology_psf_path, stride=1)    
CG_traj = md.load(CG_trajectory_data_path, top=topology_psf_path, stride=1)
CG_rdf = calculate_rdf(CG_traj,max_cutoff=16)
AA_rdf = calculate_rdf(AA_traj,max_cutoff =16)
AA_r_aa, AA_g_r_aa = AA_rdf['AA']
AA_r_bb, AA_g_r_bb = AA_rdf['BB']
AA_r_ab, AA_g_r_ab = AA_rdf['AB']

CG_r_aa, CG_g_r_aa = CG_rdf['AA']
CG_r_bb, CG_g_r_bb = CG_rdf['BB']
CG_r_ab, CG_g_r_ab = CG_rdf['AB']

In [ ]:

#figure size
plt.figure(figsize=(3, 3))
# polit A-A
plt.plot(CG_r_aa*10, CG_g_r_aa, label="CG", linestyle='-')
plt.plot(AA_r_aa*10, AA_g_r_aa, label="Atomistic", linestyle='--', color='r')
plt.xlabel(r'$r$($\AA$)', fontsize=14)
plt.ylabel("$g(r)$",fontsize=14)
plt.title("A-A")
plt.xticks(ticks=range(0, 17), labels=[str(i) if i % 5 == 0 else '' for i in range(0, 17)])
plt.legend()
plt.tight_layout()
plt.show()

# plot A-B
plt.figure(figsize=(3, 3))
plt.plot(CG_r_ab*10, CG_g_r_ab, label="CG", linestyle='-')
plt.plot(AA_r_ab*10, AA_g_r_ab, label="Atomistic", linestyle='--', color='r')
plt.xlabel(r'$r$($\AA$)', fontsize=14)
plt.ylabel("$g(r)$",fontsize=14)
plt.title("A-B")
plt.xticks(ticks=range(0, 17), labels=[str(i) if i % 5 == 0 else '' for i in range(0, 17)])
plt.legend()
plt.tight_layout()
plt.show()
# plot B-B
plt.figure(figsize=(3, 3))
plt.plot(CG_r_bb*10, CG_g_r_bb, label="CG", linestyle='-')
plt.plot(AA_r_bb*10, AA_g_r_bb, label="Atomistic", linestyle='--', color='r')
plt.xlabel(r'$r$($\AA$)', fontsize=14)
plt.ylabel("$g(r)$",fontsize=14)
plt.xticks(ticks=range(0, 17), labels=[str(i) if i % 5 == 0 else '' for i in range(0, 17)])
plt.title("B-B")
plt.legend()
plt.tight_layout()
plt.show()